In [ ]:
import requests
import tempfile
import shutil
import subprocess
import shlex
import os
import sqlite3
import pandas
import xml.etree.cElementTree as etree

# Stack Exchange Analysis

# Introduction

## Table of Contents

# Processing the Data

## Downloading the data
Our first step is to download the [data dumps](https://archive.org/details/stackexchange). Currently, the Stack Exchange Network hosts their data dumps through the [Internet Archive](https://archive.org/). Dump files are provided for each network (aside from StackOverflow for which there are multiple files) and are compressed using the 7z archive format. Thus, the code below to download and uncompress the data archive requires having the `7z` binary, acquirable through `apt-get`, `brew` and other package managers.

In [ ]:
def get_network_data(network, path=''):
    '''downloads StackExchange network data from archive.org'''

    # download archive
    url = 'https://archive.org/download/stackexchange/%s.stackexchange.com.7z' % network
    response = requests.get(url)
    
    with tempfile.NamedTemporaryFile('wb') as f:
        # copy 7z archive into the filesystem
        response.raw.decode_content = False
        f.write(response.content)
        f.flush()
        
        # create a folder to store the XML data
        path = os.path.join(path, network)
        if not os.path.exists(path):
            os.makedirs(path)

        # there are few Python 7z compatible libraries and they don't
        # work correctly with these archives (overwrite switch enabled)
        args = shlex.split('7z x %s -aoa "-o%s"' % (f.name, path))
        return subprocess.check_call(args)

In [ ]:
network = 'ai'
get_network_data(network)

Now that we've downloaded the archive, it's time to load the XML data within it into a database that can be easily queried when we're building our features. The [README](https://ia800500.us.archive.org/22/items/stackexchange/readme.txt) provided with the data dump describes the schema of each table and below we provide a visualization of these tables as well as the types of each field once we load them into our database. This visualization was created with the use of [WWW SQL Designer](https://ondras.zarovi.cz/sql/demo/). We note that the `posts` table covers both answers and questions and that `badges` are awards of a sort that users recieve for achieving certain milestones. While we deemphasize some of these tables throughout the feature engineering process, we nevertheless mention them here and process them for the sake of completeness.

![Database schema](schema.png)

Next, we create a SQLite database conforming to the above schema (duplicated below as a Python dictionary). Inspiration is taken from [zhangqiaorjc/sedumpy](https://github.com/zhangqiaorjc/sedumpy/blob/master/makedb.py).

In [ ]:
schema = {
    'badges': {
        'Id': 'INTEGER PRIMARY KEY',
        'UserId': 'INTEGER',
        'Name': 'VARCHAR(50)',
        'Date': 'DATETIME',
        'Class': 'INTEGER',
        'TagBased': 'VARCHAR(8)',
    },
    'comments': {
        'Id': 'INTEGER PRIMARY KEY',
        'PostId': 'INTEGER',
        'Score': 'INTEGER',
        'Text': 'TEXT',
        'CreationDate': 'DATETIME',
        'UserId': 'INTEGER',
    },
    'posts': {
        'Id': 'INTEGER PRIMARY KEY',
        'PostTypeId': 'INTEGER', # 1: Question, 2: Answer
        'ParentId': 'INTEGER', # present only if PostTypeId = 2
        'AcceptedAnswerId': 'INTEGER', # present only if PostTypeId = 1
        'CreationDate': 'DATETIME',
        'Score': 'INTEGER',
        'ViewCount': 'INTEGER',
        'Body': 'TEXT',
        'OwnerUserId': 'INTEGER', # present only if user has not been deleted
        'LastEditorUserId': 'INTEGER',
        'LastEditDate': 'DATETIME',
        'LastActivityDate': 'DATETIME',
        'Title': 'VARCHAT(256)',
        'Tags': 'VARCHAT(256)',
        'AnswerCount': 'INTEGER',
        'CommentCount': 'INTEGER',
        'FavoriteCount': 'INTEGER',
        'ClosedDate': 'DATETIME'
    },
    'votes': {
        'Id': 'INTEGER PRIMARY KEY',
        'PostId': 'INTEGER',
        'UserId': 'INTEGER',
        'VoteTypeId': 'INTEGER',
            # 1: AcceptedByOriginator
            # 2: UpMod
            # 3: DownMod
            # 4: Offensive
            # 5: Favorite
            # 6: Close
            # 7: Reopen
            # 8: BountyStart
            # 9: BountyClose
            # 10: Deletion
            # 11: Undeletion
            # 12: Spam
            # 13: InformModerator
        'CreationDate': 'DATETIME',
    },
    'post_history': {
        'Id': 'INTEGER PRIMARY KEY',
        'PostHistoryTypeId': 'INTEGER',
        'PostId': 'INTEGER',
        'RevisionGUID': 'VARCHAR(36)',
        'CreationDate': 'DATETIME',
        'UserId': 'INTEGER',
        'Comment': 'TEXT',
        'Text': 'TEXT'
    },
    'post_links': {
        'Id': 'INTEGER PRIMARY KEY',
        'CreationDate': 'DATETIME',
        'PostId': 'INTEGER',
        'RelatedPostId': 'INTEGER',
        'LinkTypeId': 'INTEGER'
    },
    'users': {
        'Id': 'INTEGER PRIMARY KEY',
        'Reputation': 'INTEGER',
        'CreationDate': 'DATETIME',
        'DisplayName': 'VARCHAR(50)',
        'LastAccessDate': 'DATETIME',
        'WebsiteUrl': 'VARCHAR(256)',
        'Location': 'VARCHAR(256)',
        'Age': 'INTEGER',
        'AboutMe': 'TEXT',
        'Views': 'INTEGER',
        'UpVotes': 'INTEGER',
        'DownVotes': 'INTEGER',
        'AccountId': 'INTEGER',
        'ProfileImageUrl': 'VARCHAR(256)'
    },
    'tags': {
        'Id': 'INTEGER PRIMARY KEY',
        'TagName': 'TEXT',
        'Count': 'INTEGER',
        'ExcerptPostId': 'INTEGER',
        'WikiPostId': 'INTEGER'
    }
}

# define a f
def create_table(c, name, constraints=[]):
    '''Creates a single table under the schema given its name and constraints'''
    fields = list(map(lambda x: '%s %s' % x, schema[name].items()))
    c.execute(
        'CREATE TABLE %s (%s)' % (
            name,
            ', '.join(fields + constraints)
        )
    )

def create_database(network, path='', database=None):
    '''Creates a database containing all the Stack Exchange network data'''
    
    # choose a name for the database
    if not database:
        database = '%s.db' % network
    database = os.path.join(path, database)
    
    conn = sqlite3.connect(database)
    
    c = conn.cursor()
    
    create_table(c, 'users')
    create_table(c, 'badges', ['FOREIGN KEY(UserId) REFERENCES users(Id)'])
    create_table(c, 'posts', [
        'FOREIGN KEY(AcceptedAnswerId) REFERENCES posts(Id)',
        'FOREIGN KEY(ParentId) REFERENCES posts(Id)',
        'FOREIGN KEY(OwnerUserId) REFERENCES users(Id)'
    ])
    create_table(c, 'votes', [
        'FOREIGN KEY(PostId) REFERENCES posts(Id)',
        'FOREIGN KEY(UserId) REFERENCES users(Id)'
    ])
    create_table(c, 'tags', [
        'FOREIGN KEY(WikiPostId) REFERENCES posts(Id)',
        'FOREIGN KEY(ExcerptPostId) REFERENCES posts(Id)'
    ])
    create_table(c, 'comments', [
        'FOREIGN KEY(PostId) REFERENCES posts(Id)',
        'FOREIGN KEY(UserId) REFERENCES users(Id)'
    ])
    create_table(c, 'post_history', [
        'FOREIGN KEY(PostId) REFERENCES posts(Id)',
        'FOREIGN KEY(UserId) REFERENCES users(Id)'
    ])
    create_table(c, 'post_links', [
        'FOREIGN KEY(PostId) REFERENCES posts(Id)',
    ])
    
    conn.commit()
    
    return (conn, database)


In [ ]:
conn, database = create_database(network)

We then create a series of indices to make our later queries faster,

In [ ]:
def create_indices(conn):
    '''creates indices on the tables to speed up later queries'''
    c = conn.cursor()

    c.execute('CREATE INDEX badges_idx_1 ON badges(UserId);')

    c.execute('CREATE INDEX comments_idx_1 ON comments(PostId);')
    c.execute('CREATE INDEX comments_idx_2 ON comments(UserId);')

    c.execute('CREATE INDEX post_history_idx_1 ON post_history(PostId);')
    c.execute('CREATE INDEX post_history_idx_2 ON post_history(UserId);')

    c.execute('CREATE INDEX posts_idx_1 ON posts(AcceptedAnswerId);')
    c.execute('CREATE INDEX posts_idx_2 ON posts(ParentId);')
    c.execute('CREATE INDEX posts_idx_3 ON posts(OwnerUserId);')
    c.execute('CREATE INDEX posts_idx_4 ON posts(LastEditorUserId);')

    c.execute('CREATE INDEX votes_idx_1 ON votes(PostId);')
    
    conn.commit()

In [ ]:
create_indices(conn)

Finally, we populate the database, one XML file at a time:

In [ ]:
def xml_file_name(path, table):
    '''returns the path to the XML file corresponding to a particular table'''
    return os.path.join(path, ''.join(map(str.capitalize, table.split('_'))) + '.xml')

def populate_data(conn, path):
    '''populates all the network data given a path to a folder containing the XML dump files'''
    c = conn.cursor()

    # iterate over each table
    for table in ['users', 'badges', 'posts', 'votes', 'tags', 'comments', 'post_history', 'post_links']:
        tree = etree.iterparse(xml_file_name(path, table))
        for _, record in tree:
            if record.attrib.values():
                query = 'INSERT INTO %s (%s) VALUES (%s)' % (
                    table, 
                    ', '.join(record.attrib.keys()),
                    ('?, ' * len(record.attrib.keys()))[:-2]
                )
                c.execute(query, record.attrib.values())
        conn.commit()

In [ ]:
populate_data(conn, network)

Finally, let's do a little sanity checking to ensure that we've correctly populated the database:

In [ ]:
def santity_check(conn, path):
    c = conn.cursor()
    
    entries = pandas.DataFrame(columns=['table', 'xml_entries', 'db_entries'])
    
    for table in schema.keys():
        tree = etree.iterparse(xml_file_name(path, table))
        xml_count = len(list(tree))
        db_count = c.execute('SELECT COUNT(*) FROM %s' % table).fetchone()[0]
        entries = entries.append([{'table': table, 'xml_entries': xml_count, 'db_entries': db_count}])
    
    entries['difference'] = (entries['xml_entries'] - entries['db_entries']).map(abs)
    return entries

In [ ]:
santity_check(conn, network)